In [10]:
from yahooquery import Ticker
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
# !pip install selenium
import selenium
from selenium import webdriver
import json
import time

In [11]:
class StockDetail():

    api_key = 'ADLK0ZK57SB9BJKX'

    def __init__(self, symbol):

        self.symbol = symbol
    # returns a bunch of information like a description of the company, marketcap, 52 week lows and highs, EPS, Dividend, etc.
    def get_company_overview(self):

        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        return response_json

    #retrieves current price of Stock
    def get_quote(self): 

        url = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        response_json = {x[4:].replace(' ','_'): v 
            for x, v in response_json['Global Quote'].items()}

        return response_json
    
    #monthly price data
    def get_monthly_adjusted(self):

        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if list(response_json.keys())[0] == 'Error Message':
            return "No data found."

        return response_json

    #live price data from yahooQuery
    def get_price(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.price
        df = pd.DataFrame.from_dict(raw_dict)

        return df
    
    #returns earnings trend data
    def get_earnings_trend(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.earnings_trend

        output_dict = {}
        output_dict['currentQtr'] = raw_dict[self.symbol]['trend'][0]['epsTrend']
        output_dict['nextQtr'] = raw_dict[self.symbol]['trend'][1]['epsTrend']
        output_dict['currentYr'] = raw_dict[self.symbol]['trend'][2]['epsTrend']
        output_dict['nextYr'] = raw_dict[self.symbol]['trend'][3]['epsTrend']

        return output_dict

    def get_option_chain(self, **kwargs):

        ticker = Ticker(self.symbol)
        df = ticker.option_chain

        if type(df) is str:
            if df == 'No option chain data found':  
                return "No data found."
                
        df.reset_index(inplace=True)
        df = df[df['optionType']=='puts']

        expireDates = df['expiration'].dt.strftime("%Y-%m-%d").unique()    

        df['mid'] = ( ( df['bid'] + df['ask'] ) / 2 ).round(2)
        df['timeToExpire'] = (df['expiration'] - pd.Timestamp.today()).round('1d').dt.days + 1
        df['multiplier'] = 365 / df['timeToExpire']
        df['return'] = ( df['multiplier'] * df['mid'] * 100 ) / ( df['strike'] * 100 )
        df.drop(['optionType', 'contractSymbol', 'currency', 'contractSize', 'lastTradeDate', 'impliedVolatility', 'multiplier'], axis=1, inplace=True)
        df['change'] = df['change'].round(2)
        df['percentChange'] = df['percentChange'].round(2)
        df['return'] = (df['return'] * 100).round(2)

        # Set filters
        if kwargs['expiration'] != 'all' and kwargs['expiration'] != None:
            df = df[ df['expiration'] == kwargs['expiration'] ]

        if kwargs['strikeMin'] != '' and kwargs['strikeMin'] != None:
            df = df[ df['strike'] >= int(kwargs['strikeMin']) ]

        if kwargs['strikeMax'] != '' and kwargs['strikeMax'] != None:
            df = df[ df['strike'] <= int(kwargs['strikeMax']) ]

        if kwargs['returnMin'] != '' and kwargs['returnMin'] != None:
            df = df[ df['return'] >= int(kwargs['returnMin']) ]

        if kwargs['returnMax'] != '' and kwargs['returnMax'] != None:
            df = df[ df['return'] <= int(kwargs['returnMax']) ]

        if kwargs['timeToExpireMin'] != '' and kwargs['timeToExpireMin'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMin']) ]

        if kwargs['timeToExpireMax'] != '' and kwargs['timeToExpireMax'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMax']) ]

        # Format Columns
        df['expiration'] = df['expiration'].dt.strftime("%Y-%m-%d")

        # Rename Columns
        column_rename = {
            'percentChange': '% Change',
            'openInterest': 'open Int'
        }
        df.rename(columns=column_rename, inplace=True)

        results = {}
        results['columns'] = df.columns.values
        results['expiration'] = expireDates
        results['data'] = df

        return results
    
def tradingView(
    mktCapMin = 5000000000,
    div_yield_recent = 2,
    StochK = 25,
    StochD = 25,
    macd_macd = 0,
    macd_signal = 0,
):
    filter = [
        {
            "left": "volume",
            "operation": "nempty"
        },
        {
            "left": "type",
            "operation": "in_range",
            "right": [
                "stock",
                "dr",
                "fund"
            ]
        },
        {
            "left": "subtype",
            "operation": "in_range",
            "right": [
                "common",
                "foreign-issuer",
                "",
                "etf",
                "etf,odd",
                "etf,otc",
                "etf,cfd"
            ]
        },
        {
            "left": "exchange",
            "operation": "in_range",
            "right": [
                "AMEX",
                "NASDAQ",
                "NYSE"
            ]
        },
        {
            "left": "market_cap_basic",
            "operation": "egreater",
            "right": mktCapMin
        },
        {
            "left": "is_primary",
            "operation": "equal",
            "right": True
        },
        {
            "left": "Stoch.K",
            "operation": "less",
            "right": StochK
        },
        {
            "left": "Stoch.D",
            "operation": "less",
            "right": StochD
        },
        {
            "left": "MACD.macd",
            "operation": "less",
            "right": macd_macd
        },
        {
            "left": "MACD.signal",
            "operation": "less",
            "right": macd_signal
        },
        {
            "left": "dividend_yield_recent",
            "operation": "egreater",
            "right": div_yield_recent
        },
        {
            "left": "Stoch.RSI.K",
            "operation": "less",
            "right": 25
        },
        {
            "left": "Stoch.RSI.D",
            "operation": "less",
            "right": 25
        }
    ]
    options = {
        "lang": "en"
    }
    markets = {
        "america"
    }
    symbols = {
        "query": {
            "types": []
        },
        "tickers": []
    }
    columns = [
        #"logoid",
        "name",
        "description",
        "close",
        "change",
        "change_abs",
        "Recommend.All",
        "market_cap_basic",
        "price_earnings_ttm",
        "earnings_per_share_basic_ttm",
        "sector",
        "earnings_release_date",
        "earnings_release_next_date",
        "dividend_yield_recent"
    ]
    sort = {
        "sortBy": "volume",
        "sortOrder": "desc"
    }
    range = [
        0,
        150
    ]

    post_message = {}
    post_message['filter'] = filter
    post_message['options'] = options
    post_message['symbols'] = symbols
    post_message['columns'] = columns
    post_message['sort'] = sort
    post_message['range'] = range

    payload = json.dumps(post_message)


    url = "https://scanner.tradingview.com/america/scan"
    #     payload = self.payload
    headers = {
        'authority': 'scanner.tradingview.com',
        'accept': 'text/plain, */*; q=0.01',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'dnt': '1',
        'origin': 'https//www.tradingview.com',
        'referer': 'https//www.tradingview.com/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    response_json = response.json()['data']
    if not response_json:
        return []
    df0 = pd.DataFrame.from_dict(response_json)
    df0.drop(columns=['s'], inplace=True)
    df = pd.DataFrame(df0["d"].to_list(), columns=columns)
#     display(df)

    df['change'] = df['change'].round(2)
    df['dividend_yield_recent'] = df['dividend_yield_recent'].round(2)
    df['price_earnings_ttm'] = df['price_earnings_ttm'].round(2)
    df['earnings_per_share_basic_ttm'] = df['earnings_per_share_basic_ttm'].round(2)
#     display(df)
    myListOfTickers = df['name'].to_list()
#     executionTime = (time.time() - startTime)
#     tradingViewTime = tradingViewTime + executionTime

    # return render(request, 'options/options.html', results)
    return myListOfTickers


In [3]:
#YAHOO QUERY
#returns earnings trend data 
# returns earnings trend for future dates
def get_earnings_trend(symbol):

    ticker = Ticker(symbol)
    raw_dict = ticker.earnings_trend
    display(raw_dict)
    output_dict = {}
    output_dict['currentQtr'] = raw_dict[symbol]['trend'][0]['epsTrend']
    output_dict['nextQtr'] = raw_dict[symbol]['trend'][1]['epsTrend']
    output_dict['currentYr'] = raw_dict[symbol]['trend'][2]['epsTrend']
    output_dict['nextYr'] = raw_dict[symbol]['trend'][3]['epsTrend']

    return output_dict

get_earnings_trend('AAPL')

{'AAPL': {'trend': [{'maxAge': 1,
    'period': '0q',
    'endDate': '2023-03-31',
    'growth': -0.059,
    'earningsEstimate': {'avg': 1.43,
     'low': 1.37,
     'high': 1.56,
     'yearAgoEps': 1.52,
     'numberOfAnalysts': 26,
     'growth': -0.059},
    'revenueEstimate': {'avg': 92976300000,
     'low': 91805900000,
     'high': 98836000000,
     'numberOfAnalysts': 24,
     'yearAgoRevenue': 97278000000,
     'growth': -0.044},
    'epsTrend': {'current': 1.43,
     '7daysAgo': 1.43,
     '30daysAgo': 1.43,
     '60daysAgo': 1.49,
     '90daysAgo': 1.49},
    'epsRevisions': {'upLast7days': 0,
     'upLast30days': 1,
     'downLast30days': 0,
     'downLast90days': {}}},
   {'maxAge': 1,
    'period': '+1q',
    'endDate': '2023-06-30',
    'growth': 0.033,
    'earningsEstimate': {'avg': 1.24,
     'low': 1.02,
     'high': 1.35,
     'yearAgoEps': 1.2,
     'numberOfAnalysts': 25,
     'growth': 0.033},
    'revenueEstimate': {'avg': 85545000000,
     'low': 81317700000,
  

{'currentQtr': {'current': 1.43,
  '7daysAgo': 1.43,
  '30daysAgo': 1.43,
  '60daysAgo': 1.49,
  '90daysAgo': 1.49},
 'nextQtr': {'current': 1.24,
  '7daysAgo': 1.24,
  '30daysAgo': 1.24,
  '60daysAgo': 1.25,
  '90daysAgo': 1.26},
 'currentYr': {'current': 5.96,
  '7daysAgo': 5.97,
  '30daysAgo': 5.97,
  '60daysAgo': 6.17,
  '90daysAgo': 6.21},
 'nextYr': {'current': 6.61,
  '7daysAgo': 6.6,
  '30daysAgo': 6.58,
  '60daysAgo': 6.74,
  '90daysAgo': 6.77}}

# YAHOO QUERY START OF SELENIUM Premium HISTORICAL EARNINGS

##### documentation for yahoo query premium
https://yahooquery.dpguthrie.com/guide/ticker/premium/#p_get_financial_data

In [12]:
!pip install yahooquery[premium]
# !pip uninstall selenium
!pip install selenium==3.5.0
import selenium
from selenium import webdriver
from yahooquery import Ticker

yf_username = 'qcappartners@yahoo.com'

yf_password = 'Mcci2022!!'
# ticker = Ticker('aapl', username= yf_username,
#     password= yf_password)
symbols = ['aapl']

faang = Ticker(symbols, asynchronous=True)
faang.login(username=yf_username, password=yf_password)


  Using cached selenium-4.8.2-py3-none-any.whl (6.9 MB)
  Attempting uninstall: selenium
    Found existing installation: selenium 3.5.0
    Uninstalling selenium-3.5.0:
      Successfully uninstalled selenium-3.5.0
  Using cached selenium-3.5.0-py2.py3-none-any.whl (921 kB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.2
    Uninstalling selenium-4.8.2:
      Successfully uninstalled selenium-4.8.2


TypeError: __init__() got an unexpected keyword argument 'options'

In [14]:
# drilldown = Ticker(symbols, username=yf_username, password=yf_password).p_value_analyzer_drilldown
# display(drilldown)
yf_username = 'qcappartners@yahoo.com'
yf_password = 'Mcci2022!!'

tickers = ['aapl', 'nflx']
# 'DilutedEPS', 'BasicEPS', 
fin_types = ['DilutedEPS', 'BasicEPS']


financial_data = Ticker(tickers, username=yf_username, password=yf_password).p_get_financial_data(types=fin_types, trailing=False)
display(financial_data)


# aapl.p_value_analyzer_drilldown
# Ticker('aapl').p_value_analyzer_drilldown

TypeError: __init__() got an unexpected keyword argument 'options'

In [ ]:
global AlphaVantageKey
#basic key AlphaVantageKey = 'XEQ9AFU8KM035KMG'
#premium api key
AlphaVantageKey = 'ZTB6U564ILR50HU3'
# @cache_request()
def fairValue_hist(ticker):
    global AlphaVantageKey

    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retriving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='7y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        display(priceHistory)
#         priceHistory['date'] = priceHistory['date'].dt.strftime('%Y-%m-%d')
#         print('dtypes')
        display(priceHistory.dtypes)
        priceHistory.drop(priceHistory.tail(1).index,inplace=True)
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

#     print('fv_df_currentYR: ', fv_df)
    return fv_df


fairValue_hist('aapl')

tvList = tradingView()
print(len(tvList))
display(tvList)
def earningsUpdate(tickerList):
    
    yf_username = 'qcappartners@yahoo.com'
    yf_password = 'Mcci2022!!'

    tickers = tickerList
    # 'DilutedEPS', 'BasicEPS', 
    fin_types = ['DilutedEPS', 'BasicEPS']
    financial_data = Ticker(tickers, username=yf_username, password=yf_password).p_get_financial_data(types=fin_types, trailing=False)
    financial_data = financial_data.reset_index()
    financial_data.to_csv('earnings.csv', index=False)
    display(financial_data)

earningsUpdate(tvList)



In [ ]:

def fairValue_hist(ticker):
    global AlphaVantageKey

    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    
    ##THIS IS THE ALPHA VANTAGE EARNINGS CALL WHICH IS BEING DEPRECATED TO USE YAHOOQUERY
#     url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
#     r = requests.get(url)
#     data = r.json()
#     display(data)
#     df = pd.DataFrame(data['annualEarnings'])
#     print('earnings df alpha')
#     display(df)
    
    
    earningsDF = pd.read_csv('earnings.csv')
    earningsDF = earningsDF[earningsDF['symbol'] == ticker]
    df = pd.DataFrame()
    df['fiscalDateEnding'] = earningsDF['asOfDate']
    df['reportedEPS'] = earningsDF['DilutedEPS']
    # fiscalDateEnding reportedEPS
#     print('yq earnings')
    
#     df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retriving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='7y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
#         display(priceHistory)
#         priceHistory['date'] = priceHistory['date'].dt.strftime('%Y-%m-%d')
#         print('dtypes')
#         display(priceHistory.dtypes)
        priceHistory.drop(priceHistory.tail(1).index,inplace=True)
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

#     print('fv_df_currentYR: ', fv_df)
    return fv_df

# theTicker = 'XOM'

# earningsDF = pd.read_csv('earnings.csv')

# earningsDF = earningsDF[earningsDF['symbol'] == theTicker]
# df = pd.DataFrame()
# df['fiscalDateEnding'] = earningsDF['asOfDate']
# df['reportedEPS'] = earningsDF['DilutedEPS']
# # fiscalDateEnding reportedEPS
# print('yq earnings')
# display(df)


display(fairValue_hist(theTicker))